# In this assignment, a simple ConvNet is implemented and trained to predict the language of a surname from its spelling in English.

In [17]:
import os
import numpy as np
from tqdm import tqdm

## 1. ConvNet class implementation

In [56]:
class ConvNet:
    """
    A simple ConvNet implementation with 2 convolutional layers 
    followed by a fully connected layer and a softmax output 
    layer.
    
    Attributes:
        n: A list of the number of filters applied at each convo-
            lutional layer.
        k: A list of the width of the filter window applied in each
            convolutional layer.
        input_dim: dimensionality of input
        output_dim: dimensionality of output
        nlen_list: List with the number of columns of the input when in its original form
            before being vectorized, for each layer.
        eta: initial learning rate value
        rho: momentum constant term
        F: list of filters/weights for each one of the convolutional
            layers.
        W: Weights of the fully connected layer
        
    """
    
    def __init__(self, n, k, output_dim, input_dim, nlen, eta = 0.001, rho = 0.95, he_init = False):
        """
        Initializes the convolutional neural network
    
        Args:
            he_init: When True He-initialization is performed for the weights
            nlen: Number of columns of the input when in its original form
            before being vectorized.
        Raises:
            Exception if the size of n and k is not the same.
        """
        if(len(n) != len(k)):
            raise Exception("The number of layers specified by <n> " \
                            "and <k> should be equal.")
        self.n = list(n)
        self.k = list(k)
        
        self.input_dim = input_dim
        
        self.F = []
        self.F.append(np.random.randn(input_dim, self.k[0], self.n[0]))
        self.nlen_list = [nlen - self.k[0] + 1]
        for i in range(len(n)):
            self.F.append(np.random.randn(self.n[i - 1], self.k[i], self.n[i]))
            self.nlen_list.append(self.nlen_list[-1] - self.k[1] + 1)
        fsize = self.n[-1] * self.nlen_list[-1]
        self.W = np.random.randn(output_dim, fsize)

## 2. Preparing data

In [19]:
def encodeString(string, character_dictionary, max_length):
    """
    One-hot encodes the character string, converting each 
    of its letters to one-hot encoded vectors and stacking
    them from left to right. 
    
    Args:
        name: The string to be encoded.
        character_dictionary: A dictionary which has a unique
            index for each character in the alphabet used by
            the string.
        max_length: maximum length of the string. If the string
            has a length less than max_length, zero columnds are
            added as padding after the encoded character columns.

    
    Returns:
        A C x max_length vector with the one-hot encoded characters
        of the string and possibly zero padding in the last columns
        where C is the number of different characters in the alpha-
        bet used.
    """
    d = len(character_dictionary)
    encoded_string = np.zeros((d, max_length))
    for i in range(len(string)):
        encoded_string[character_dictionary[string[i]],i] = 1
    return encoded_string

In [20]:
# Path to the files containing the data
name_path = "ascii_names.txt"
category_labels_path = "category_labels.txt"
# Path to file used to save the inputs after their encoding
save_input_path = "onehot_encoded_inputs.npy"
# Path to file with the indices of the inputs that are going to used in the validation set
val_ind_path = "Validation_Inds.txt"

In [21]:
# Read the data
names = []
labels = []
if(os.path.exists(name_path)):
    with open(name_path,"r") as f:
        for line in tqdm(f):
            entry = line.split()
            names.append(' '.join(entry[:-1]))
            labels.append(entry[-1])
    f.close()
    names = np.array(names)
    labels = np.array(labels, dtype = int) 
else:
    print("Requested file " + name_path + " does not exist.")

20050it [00:00, 1002297.83it/s]


In [22]:
# Read the different class names and indices and build a dictionary
if(os.path.exists(category_labels_path)):
    class_names = np.loadtxt(category_labels_path, usecols = 1, dtype = str)
    class_indices = np.loadtxt(category_labels_path, usecols = 0, dtype = int)
    K = len(class_names)
    class_dictionary = {}
    for i in range(K):
        class_dictionary[class_names[i]] = class_indices[i]
    inv_class_dictionary = {v: k for k, v in class_dictionary.items()}
    # Check for correctness
    print(class_dictionary['Arabic'])
    print(inv_class_dictionary[1])
else: 
    print("Requested file " + category_labels_path + " does not exist.")

1
Arabic


### Determine number of unique characters and set up dictionary / Determine maximum length of name in dataset

In [23]:
character_dictionary = {}
unique_idx = 0
max_length = -1
for name in tqdm(names):
    length = len(name)
    if(length > max_length):
        max_length = length
    for i in range(len(name)):
        if(name[i] not in character_dictionary.keys()):
            character_dictionary[name[i]] = unique_idx
            unique_idx += 1
d = len(character_dictionary) # Get number of unique characters

100%|████████████████████████████████| 20050/20050 [00:00<00:00, 489079.75it/s]


In [24]:
print("DIFFERENT UNIQUE CHARACTERS: " + str(d))
print("MAXIMUM NAME LENGTH: " + str(max_length))

DIFFERENT UNIQUE CHARACTERS: 55
MAXIMUM NAME LENGTH: 19


In [25]:
# Build inverse dictionary mapping
inv_character_dictionary = {v: k for k, v in character_dictionary.items()}
# Check for correctness
print(character_dictionary['o'])
print(inv_character_dictionary[2])

2
o


### One-hot encoding and vectorization of the input names

In [26]:
# Encode and save the inputs in a matrix when each column corresponds to a different name
vectorized_input_size = d * max_length
X = np.zeros((vectorized_input_size, names.shape[0]))
for idx, name in enumerate(tqdm(names)):
    X[:,idx] = encodeString(name, character_dictionary, max_length).flatten(order = 'F')

100%|█████████████████████████████████| 20050/20050 [00:00<00:00, 32786.18it/s]


In [27]:
# Save inputs in a file if they are not already saved
if(not os.path.exists(save_input_path)):
    np.save(save_input_path, X)

In [28]:
# Get the indices of the inputs that are going to used in the validation set
if(os.path.exists(val_ind_path)):
    validation_indices = np.loadtxt(val_ind_path)

## Set hyperparameters & initialize the ConvNet's parameters

In [57]:
filter_sizes = [5, 5]
filter_width_constants = [3, 3]
K = len(class_dictionary)
conv_net = ConvNet(n = filter_sizes , k = filter_width_constants, output_dim = K, input_dim = d, nlen = max_length)

In [58]:
print(conv_net.F[0].shape)
print(conv_net.F[1].shape)
print(conv_net.W.shape)

(55, 3, 5)
(5, 3, 5)
(18, 65)
